# 写在前面：数据总结--猎聘网

**☞项目名称：** UI精准在猎聘

**☞数据加值宣言：**本项目产出按**热门公司、行业以及年薪**挖掘的关于**UI设计师**工作的分页数据，以解决UI设计师就业需求及特性的就业分析问题

**☞MVP数据加值：** 【此产品面向对象为：UI设计师】【为求职者提供按个人偏好来筛选工作的可能】

                1、满足公司、行业、薪资三种限制的分页数据，解决UI设计师对公司、行业类别和薪资档位的多种就业需求
                2、以公司为分类的多页数据（如：中国500强、上市公司等分类）解决UI设计师对公司类别的单一就业需求
                3、以行业为分类的多页数据（如：互联网/电商、通信业等分类）解决UI设计师对行业类别的单一就业需求
                4、以薪资为分类的多页数据（如：10-15w、15-20w等分类）解决UI设计师对薪资档位的单一就业需求
                
**☞Query参数：** 
                
                1、公司类别：compTag
                2、行业类别：industryType、industries
                3、薪资类别：salary
                4、翻页：curPage
                

以下的数据<br>整体目录如下：

- ①必用
- ②参数模板公司
- ③参数模板行业
- ④参数模板薪资
- ⑤参数模板翻页
- ⑥基础页面
- ⑦页面-分类-公司
- ⑧页面-分类-行业
- ⑨页面-分类-薪资
- ⑩页面-分类-翻页
- ⑪汇总以上数据价值表格

# ①必用

In [1]:
# 导入模块
import pandas as pd
from requests_html import HTMLSession
from urllib.parse import urlparse, parse_qs
from IPython.display import display, HTML
import time
from random import random

In [2]:
# 主要链接
url = "https://www.liepin.com/zhaopin/?key=UI"
session = HTMLSession()
r = session.get( url )

# ②参数模板公司

In [3]:
#A ❶建构参数模板-公司

#获取公司类别的xpath
company_select = r.html.xpath('//div[@data-selector="search-conditions"]')[0] \
                    .xpath('//dt[@class="search-title"]/following-sibling::dd')[0] \
                    .xpath('//div[contains(@class,"hot-comp-tags")]/a')

company_select = { x.xpath("a/text()")[0]:x.xpath("a/@href")[0] for x in company_select}
#print(company_select)

#封装解析query的函数
def parse_url_qs_for_compTag (url):
    six_parts = urlparse(url) 
    out = parse_qs(six_parts.query)
    return (out)

df = pd.DataFrame([ urlparse(x) for x in company_select.values()])
df_qs = pd.DataFrame([{k:v[0] for k,v in parse_qs(x).items()} for x in df['query'] ])
print (df_qs.nunique())
df_qs.head()
display(df_qs[['key','compTag']]) #发现comTag对应到不同类型的公司

#参数模板
company_params = parse_url_qs_for_compTag(list(company_select.values())[0])
#print(company_params)

#字典:comTag
company_compTag = { k:parse_url_qs_for_compTag(v)['compTag'][0] for k,v in company_select.items()}
#print (company_compTag)

def make_parameter(compTag , keyword ):
    params = company_params.copy()
    params['compTag'] = compTag
    params['keyword'] = keyword
    return (params)

company_compTag_UI = { k:make_parameter(compTag = [v], keyword = ['UI']) for k,v in company_compTag.items()}
print(company_compTag_UI)

init             1
headckid         1
flushckid        1
fromSearchBtn    1
compTag          6
ckid             1
key              1
siTag            1
d_sfrom          1
d_ckId           1
d_curPage        1
d_pageSize       1
d_headId         1
dtype: int64


,key,compTag
0,UI,155
1,UI,182
2,UI,186
3,UI,189
4,UI,130
5,UI,156


{'中国500强': {'init': ['-1'], 'headckid': ['a0986a39c4bcdd91'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'compTag': ['155'], 'ckid': ['a0986a39c4bcdd91'], 'key': ['UI'], 'siTag': ['cf9xUm0V24brUPysJF0YOw~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_unknown'], 'd_ckId': ['f35e161ececd52aefa8437812a616fd0'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['f35e161ececd52aefa8437812a616fd0'], 'keyword': ['UI']}, '2018互联网300强': {'init': ['-1'], 'headckid': ['a0986a39c4bcdd91'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'compTag': ['182'], 'ckid': ['a0986a39c4bcdd91'], 'key': ['UI'], 'siTag': ['cf9xUm0V24brUPysJF0YOw~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_unknown'], 'd_ckId': ['f35e161ececd52aefa8437812a616fd0'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['f35e161ececd52aefa8437812a616fd0'], 'keyword': ['UI']}, '制造业500强': {'init': ['-1'], 'headckid': ['a0986a39c4bcdd91'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'compTag': ['186'], 'ckid': ['a0986a39c4bcdd91

# ③参数模板行业

In [4]:
#A ❷建构参数模板-行业

#获取行业类别的xpath
industry_select = r.html.xpath('//div[@data-selector="search-conditions"]')[0] \
                    .xpath('//dt[@class="search-title"]/following-sibling::dd[@class="short-dd select-industry"]')[0] \
                    .xpath('//div[@class="sub-industry"]//a')

industry_select = { x.xpath("a/text()")[0]:x.xpath("a/@href")[0] for x in industry_select}
#print(industry_select)

#封装解析query的函数
def parse_url_qs_for_industry(url):
    six_parts = urlparse(url) 
    out = parse_qs(six_parts.query)
    return (out)

df = pd.DataFrame([ urlparse(x) for x in industry_select.values()])
df_qs = pd.DataFrame([{k:v[0] for k,v in parse_qs(x).items()} for x in df['query'] ])
print(df_qs.nunique())
df_qs.head()
display(df_qs[['key','industryType','industries']]) #发现industryType、industries两种参数可对应，下面选其一

#参数模板
industry_params = parse_url_qs_for_industry(list(industry_select.values())[0])
#print(industry_params)

#字典:industries
industry_industries = { k:parse_url_qs_for_industry(v)['industries'][0] for k,v in industry_select.items()}
#print (industry_industries)

def make_parameter(industries,keyword):
    params = industry_params.copy()
    params['industries'] = industries
    params['keyword'] = keyword
    return (params)

industry_params_UI = { k:make_parameter(industries=[v],keyword = ['UI']) for k,v in industry_industries.items()}
print(industry_params_UI)

init              1
industryType     12
headckid          1
flushckid         1
fromSearchBtn     1
industries       51
ckid              1
key               1
siTag             1
d_sfrom           1
d_ckId            1
d_curPage         1
d_pageSize        1
d_headId          1
dtype: int64


,key,industryType,industries
0,UI,industry_01,040
1,UI,industry_01,420
2,UI,industry_01,010
3,UI,industry_01,030
4,UI,industry_02,050
5,UI,industry_02,060
6,UI,industry_02,020
7,UI,industry_03,080
8,UI,industry_03,100
9,UI,industry_03,090


{'互联网/电商': {'init': ['-1'], 'industryType': ['industry_01'], 'headckid': ['a0986a39c4bcdd91'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'industries': ['040'], 'ckid': ['a0986a39c4bcdd91'], 'key': ['UI'], 'siTag': ['cf9xUm0V24brUPysJF0YOw~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_unknown'], 'd_ckId': ['f35e161ececd52aefa8437812a616fd0'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['f35e161ececd52aefa8437812a616fd0'], 'keyword': ['UI']}, '游戏产业': {'init': ['-1'], 'industryType': ['industry_01'], 'headckid': ['a0986a39c4bcdd91'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'industries': ['420'], 'ckid': ['a0986a39c4bcdd91'], 'key': ['UI'], 'siTag': ['cf9xUm0V24brUPysJF0YOw~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_unknown'], 'd_ckId': ['f35e161ececd52aefa8437812a616fd0'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['f35e161ececd52aefa8437812a616fd0'], 'keyword': ['UI']}, '计算机软件': {'init': ['-1'], 'industryType': ['industry_01'], 'headckid': ['a0986a39c4bcdd

# ④参数模板薪资

In [70]:
#A ❸建构参数模板-薪资

#获取薪资的xpath
salary_select = r.html.xpath('//div[@data-selector="search-conditions"]')[0] \
                    .xpath('//dt[@class="search-title"]/following-sibling::dd[@data-param="salary"]/a')
salary_select = { x.xpath("a/text()")[0]:x.xpath("a/@href")[0] for x in salary_select}
#print(salary_select)

#封装解析query的函数
def parse_url_qs_for_salary(url):
    six_parts = urlparse(url) 
    out = parse_qs(six_parts.query)
    return (out)

df = pd.DataFrame([ urlparse(x) for x in salary_select.values()])
df_qs = pd.DataFrame([{k:v[0] for k,v in parse_qs(x).items()} for x in df['query'] ])
print(df_qs.nunique())
df_qs.head()
display(df_qs[['key','salary']])#发现salary可对应

#参数模板
salary_params = parse_url_qs_for_salary(list(salary_select.values())[0])
print(salary_params)

#字典:salary
salary = { k:parse_url_qs_for_salary(v)['salary'][0] for k,v in salary_select.items()}
#print (salary)

def make_parameter(salary,keyword):
    params = salary_params.copy()
    params['salary'] = salary
    params['keyword'] = keyword
    return (params)

salary_params_UI = { k:make_parameter(salary=[v],keyword = ['UI']) for k,v in salary.items()}
print(salary_params_UI)

init             1
headckid         1
flushckid        1
fromSearchBtn    1
salary           6
ckid             1
curPage          1
keyword          1
key              1
siTag            1
d_sfrom          1
d_ckId           1
d_curPage        1
d_pageSize       1
d_headId         1
dtype: int64


,key,salary
0,UI,10$15
1,UI,15$20
2,UI,20$30
3,UI,30$50
4,UI,50$100
5,UI,100$999


{'init': ['-1'], 'headckid': ['a0986a39c4bcdd91'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'salary': ['10$15'], 'ckid': ['a0986a39c4bcdd91°radeFlag=0'], 'curPage': ['9'], 'keyword': ['UI'], 'key': ['UI'], 'siTag': ['1CzcFa5S25l4xZFiV9WBNw~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_prime'], 'd_ckId': ['c62199da018313243705b4665a01aba3'], 'd_curPage': ['9'], 'd_pageSize': ['40'], 'd_headId': ['c62199da018313243705b4665a01aba3']}
{'10-15万': {'init': ['-1'], 'headckid': ['a0986a39c4bcdd91'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'salary': ['10$15'], 'ckid': ['a0986a39c4bcdd91°radeFlag=0'], 'curPage': ['9'], 'keyword': ['UI'], 'key': ['UI'], 'siTag': ['1CzcFa5S25l4xZFiV9WBNw~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_prime'], 'd_ckId': ['c62199da018313243705b4665a01aba3'], 'd_curPage': ['9'], 'd_pageSize': ['40'], 'd_headId': ['c62199da018313243705b4665a01aba3']}, '15-20万': {'init': ['-1'], 'headckid': ['a0986a39c4bcdd91'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'salary

# ⑤参数模板翻页

In [6]:
#A ❹建构参数模板-翻页

#获取翻页的xpath
xpath_翻页a = '//div[@class="pagerbar"]/a[starts-with(@href,"/zhaopin")]'
                                       # [starts-with(@href,"/zhaopin")]   
                                      #  以...开头的(通常这里是属性,"链接")
href_字典 = {x.text:x.xpath('//@href')[0]  for x in r.html.xpath(xpath_翻页a)}
#print (href_字典)
href_列表 = [x.xpath('//@href')[0] for x in r.html.xpath(xpath_翻页a)]
#print (href_列表)

#封装解析query的函数
def parse_url_qs_for_curPage (url):
    six_parts = urlparse(url) 
    out = parse_qs(six_parts.query)
    return (out)

df = pd.DataFrame([ urlparse(x) for x in href_列表])
df_aa = pd.DataFrame([{k:v[0] for k,v in parse_qs(x).items()} for x in df['query'] ])
df_aa = df_aa.assign (curPage_int=df_aa.curPage.astype(int)) 
#display(df)
#display(df_aa)
print(df_aa.nunique())

#参数模板
curPage_params = parse_url_qs_for_curPage(href_列表[0]) 
#print ("curPage:",curPage_params)

def 参数模板生成(keyword, curPage):
    参数 = curPage_params.copy()
    参数['curPage'] = curPage
    参数['keyword'] = keyword
    return (参数)

参数_keyword_UI_curPage = { 
    i:参数模板生成(curPage = [i], \
                  keyword = ['UI']) \
    for i,v in href_字典.items()\
    }

# \反斜杠：续行符（在行尾时）
# print(参数_keyword_UI_curPage) # 结果显示：curPage=2 一直到 下一页
                                      # 只生成本页有的额外翻页URL, 并没有推估到&curPage=9,也没有这页

print (df_aa.curPage_int.min()) # 最小值只有1
print (df_aa.curPage_int.max()) # 最大值只有9

# 应该是 0 (本页)....9(最大值)

参数_keyword_UI_curPage = { 
    i:参数模板生成(curPage = [i], \
                  keyword = ['UI']) \
    for i in range(0,df_aa.curPage_int.max()+1)\
    }
参数_keyword_UI_curPage

init             1
headckid         1
fromSearchBtn    1
ckid             1
key              1
siTag            1
d_sfrom          1
d_ckId           1
d_curPage        1
d_pageSize       1
d_headId         1
curPage          5
curPage_int      5
dtype: int64
1
9


{0: {'init': ['-1'],
  'headckid': ['a0986a39c4bcdd91'],
  'fromSearchBtn': ['2'],
  'ckid': ['a0986a39c4bcdd91°radeFlag=0'],
  'key': ['UI'],
  'siTag': ['cf9xUm0V24brUPysJF0YOw~fA9rXquZc5IkJpXC-Ycixw'],
  'd_sfrom': ['search_unknown'],
  'd_ckId': ['f35e161ececd52aefa8437812a616fd0'],
  'd_curPage': ['0'],
  'd_pageSize': ['40'],
  'd_headId': ['f35e161ececd52aefa8437812a616fd0'],
  'curPage': [0],
  'keyword': ['UI']},
 1: {'init': ['-1'],
  'headckid': ['a0986a39c4bcdd91'],
  'fromSearchBtn': ['2'],
  'ckid': ['a0986a39c4bcdd91°radeFlag=0'],
  'key': ['UI'],
  'siTag': ['cf9xUm0V24brUPysJF0YOw~fA9rXquZc5IkJpXC-Ycixw'],
  'd_sfrom': ['search_unknown'],
  'd_ckId': ['f35e161ececd52aefa8437812a616fd0'],
  'd_curPage': ['0'],
  'd_pageSize': ['40'],
  'd_headId': ['f35e161ececd52aefa8437812a616fd0'],
  'curPage': [1],
  'keyword': ['UI']},
 2: {'init': ['-1'],
  'headckid': ['a0986a39c4bcdd91'],
  'fromSearchBtn': ['2'],
  'ckid': ['a0986a39c4bcdd91°radeFlag=0'],
  'key': ['UI'],
  'si

# ⑥基础页面

In [7]:
# 创建页面请求函数
def requests_liepin( url, params):
    r = session.get( url , params = payload)
    #主要的xpath
    main = r.html.xpath( '//ul[@class="sojob-list"]/li')
    #具体的xpath
    dict_xpaths={ 
        'text': {
            '学历要求':      '//div[contains(@class,"job-info")]/p/span[@class="edu"]',
            '工作经验':      '//div[contains(@class,"job-info")]/p/span[@class="edu"]/following-sibling::span',
            '薪资':    '//div[contains(@class,"job-info")]/p/span[@class="text-warning"]', 
            '发布时间':    '//div[contains(@class,"job-info")]/p/time/@title', 
            '职位名称':    '//div[contains(@class,"job-info")]/h3/a', 
            '公司地点': '//div[contains(@class,"job-info")]/p/a',
            '公司名称': '//div[contains(@class,"sojob-item-main")]//p[@class="company-name"]/a', 
        },
        'text_content': {
        },
        'href': {
            '职位URL':    '//div[contains(@class,"job-info")]/h3/a', 
            '公司URL': '//div[contains(@class,"sojob-item-main")]//p[@class="company-name"]/a', 
        }
    }

    def get_e_text_content(_xpath_):
        暂存结果 = [e.xpath(_xpath_)[0].lxml.text_content() for e in main]
        return(暂存结果)

    def get_e_text(_xpath_):
        暂存结果 = ["".join([x.strip() if type(x) is str else x.text.strip() for x in e.xpath(_xpath_)]) for e in main]
        return(暂存结果)

    def get_e_href(_xpath_):
        暂存结果 = [list(e.xpath(_xpath_, first=True).absolute_links)[0] \
                   if len(e.xpath(_xpath_, first=True).absolute_links) >= 1  \
                   else "" for e in main]
        return(暂存结果)

    # 只对主要元素下进行.xpath取值
    数据字典 = dict()

    数据字典 = {k:get_e_text_content(v) for k,v in dict_xpaths['text_content'].items()}
    数据字典.update({k:get_e_text(v) for k,v in dict_xpaths['text'].items()})
    数据字典.update({k:get_e_href(v) for k,v in dict_xpaths['href'].items()})

    数据 = pd.DataFrame(数据字典)
    return (数据)

# ⑦页面-分类-公司

In [57]:
%%time
# 页面-参数模板公司
r = session.get( url, params = payload)
list_df = list()
for k,v in company_compTag_UI.items():
    payload = v
    df = requests_liepin( url, params = payload)
    time.sleep(3+4*random())
    df = df.assign (搜索关键词 = key)
    df = df.assign (公司类型 = k)    
    list_df.append(df)

df_all_company = pd.concat(list_df)
display(df_all_company)

#输出换index
df_all_company_2 = pd.concat(list_df).reset_index()
df_all_company_2.index = range(1,len(df_all_company) + 1)
df_all_company_2.index.name = '序'

#df_all_company_2.to_excel("公司分类_UI_猎聘.xlsx",\
#               sheet_name="_".join(keywords))

,学历要求,工作经验,薪资,发布时间,职位名称,公司地点,公司名称,职位URL,公司URL,搜索关键词,公司类型
0,本科及以上,1-3年,面议,2020年04月26日,29911-UI工程师,北京,腾讯,https://www.liepin.com/job/1927757849.shtml,https://www.liepin.com/company/7983148/,UI,中国500强
1,本科及以上,3-5年,面议,2020年04月26日,29923-高级UI设计师（上海）,上海,腾讯,https://www.liepin.com/job/1927657275.shtml,https://www.liepin.com/company/7983148/,UI,中国500强
2,本科及以上,1-3年,面议,2020年04月26日,HY6-UI开发(重构),深圳,腾讯,https://www.liepin.com/job/1927497127.shtml,https://www.liepin.com/company/7983148/,UI,中国500强
3,本科及以上,3-5年,面议,2020年04月26日,15616-高级UI设计师（上海）,上海,腾讯,https://www.liepin.com/job/1927497099.shtml,https://www.liepin.com/company/7983148/,UI,中国500强
4,本科及以上,3-5年,面议,2020年04月26日,15618-UI视觉设计师,上海,腾讯,https://www.liepin.com/job/1927414925.shtml,https://www.liepin.com/company/7983148/,UI,中国500强
...,...,...,...,...,...,...,...,...,...,...,...
35,本科及以上,3-5年,面议,2020年04月26日,UI平面设计师,上海-长宁区,安正时尚,https://www.liepin.com/job/1926027699.shtml,https://www.liepin.com/company/943791/,UI,上市公司
36,本科及以上,经验不限,14-20k·13薪,2020年04月26日,UI设计师（视觉组） (MJ002644),广州,欢聚集团,https://www.liepin.com/job/1926010099.shtml,https://www.liepin.com/company/930104/,UI,上市公司
37,大专及以上,5-10年,面议,2020年04月26日,15712-Visual Artist for Game UI(Los Angeles),深圳,腾讯,https://www.liepin.com/job/1926008429.shtml,https://www.liepin.com/company/7983148/,UI,上市公司
38,本科及以上,5-10年,面议,2020年04月26日,15569-QQ飞车高级视觉UI设计师,深圳,腾讯,https://www.liepin.com/job/1925864395.shtml,https://www.liepin.com/company/7983148/,UI,上市公司


Wall time: 34.3 s


# ⑧页面-分类-行业

In [58]:
%%time
# 页面-参数模板行业
r = session.get( url, params = payload)
list_df = list()
for k,v in industry_params_UI.items():
    payload = v
    df = requests_liepin( url, params = payload)
    time.sleep(3+4*random())
    df = df.assign (搜索关键词 = key)
    df = df.assign (行业类型 = k)    
    list_df.append(df)

df_all_industry = pd.concat(list_df)
display(df_all_industry)

#输出换index
df_all_industry_2 = pd.concat(list_df).reset_index()
df_all_industry_2.index = range(1,len(df_all_industry) + 1)
df_all_industry_2.index.name = '序'

#df_all_industry_2.to_excel("行业分类_UI_猎聘.xlsx",\
#                sheet_name="_".join(keywords))

,学历要求,工作经验,薪资,发布时间,职位名称,公司地点,公司名称,职位URL,公司URL,搜索关键词,行业类型
0,大专及以上,5-10年,20-30k·14薪,2020年04月24日,游戏主UI设计,上海,恺英网络,https://www.liepin.com/job/1927570189.shtml,https://www.liepin.com/company/5534889/,UI,互联网/电商
1,大专及以上,3-5年,15-20k·12薪,2020年04月26日,UI设计师,北京,触控,https://www.liepin.com/job/1927784615.shtml,https://www.liepin.com/company/842555/,UI,互联网/电商
2,本科及以上,1-3年,7-9k·12薪,2020年04月26日,UI交互设计工程师,南昌-南昌县,江西省智能产业技术创新研究院,https://www.liepin.com/job/1927784253.shtml,https://www.liepin.com/company/12139175/,UI,互联网/电商
3,统招本科,3-5年,10-17k·13薪,2020年04月26日,UI设计师,广州-番禺区,广东印萌科技有限公司,https://www.liepin.com/job/1927782097.shtml,https://www.liepin.com/company/12223213/,UI,互联网/电商
4,大专及以上,1-3年,8-24k·13薪,2020年04月26日,UI界面设计师,广州,广州多益网络股份有限公司,https://www.liepin.com/job/1927779859.shtml,https://www.liepin.com/company/8142995/,UI,互联网/电商
...,...,...,...,...,...,...,...,...,...,...,...
32,学历不限,经验不限,3-5k·12薪,2019年06月10日,UI设计师,保定,安国市华海房地产开发有限公司,https://www.liepin.com/job/1913771343.shtml,https://www.liepin.com/company/9536594/,UI,其他行业
33,本科及以上,3-5年,10-18k·12薪,2019年05月18日,UI设计产品经理,济南,山东易通发展集团有限公司,https://www.liepin.com/job/1919992965.shtml,https://www.liepin.com/company/8982720/,UI,其他行业
34,大专及以上,3-5年,10-18k·12薪,2019年04月24日,高级UI设计师,深圳,深圳市爱都科技有限公司,https://www.liepin.com/job/1919421941.shtml,https://www.liepin.com/company/9678561/,UI,其他行业
35,大专及以上,1-3年,4-8k·12薪,2019年04月17日,UI网页设计师前端,常州,江苏上觉文化传播有限公司,https://www.liepin.com/job/1917738679.shtml,https://www.liepin.com/company/9528391/,UI,其他行业


Wall time: 4min 32s


# ⑨页面-分类-薪资

In [59]:
%%time
# 页面-参数模板薪资
r = session.get( url, params = payload)
list_df = list()
for k,v in salary_params_UI.items():
    payload = v
    df = requests_liepin( url, params = payload)
    time.sleep(3+4*random())
    df = df.assign (搜索关键词 = key)
    df = df.assign (薪资类型 = k)    
    list_df.append(df)

df_all_salary = pd.concat(list_df)
display(df_all_salary)

#输出换index
df_all_salary_2 = pd.concat(list_df).reset_index()
df_all_salary_2.index = range(1,len(df_all_salary) + 1)
df_all_salary_2.index.name = '序'

#df_all_salary_2.to_excel("薪资分类_UI_猎聘.xlsx",\
#                sheet_name="_".join(keywords))

,学历要求,工作经验,薪资,发布时间,职位名称,公司地点,公司名称,职位URL,公司URL,搜索关键词,薪资类型
0,大专及以上,经验不限,6-10k·12薪,2020年04月26日,UI设计,上海,艾瑞碧(上海)化妆品有限公司,https://www.liepin.com/job/1927592063.shtml,https://www.liepin.com/company/10197959/,UI,10-15万
1,本科及以上,1-3年,6-10k·12薪,2020年04月23日,UI设计师,深圳,深圳市兰兹酒店管理有限公司,https://www.liepin.com/job/1927021149.shtml,https://www.liepin.com/company/9706919/,UI,10-15万
2,本科及以上,3-5年,面议,2020年04月26日,市场销售支持专员(UI&UE设计),上海-闵行区,观致汽车有限公司,https://www.liepin.com/job/1927786173.shtml,https://www.liepin.com/company/3166253/,UI,10-15万
3,统招本科,3-5年,7-10k·14薪,2020年04月26日,UI设计讲师,大连-凌水,成都中慧科技有限公司,https://www.liepin.com/job/1927784901.shtml,https://www.liepin.com/company/12160109/,UI,10-15万
4,本科及以上,1-3年,7-9k·12薪,2020年04月26日,UI交互设计工程师,南昌-南昌县,江西省智能产业技术创新研究院,https://www.liepin.com/job/1927784253.shtml,https://www.liepin.com/company/12139175/,UI,10-15万
...,...,...,...,...,...,...,...,...,...,...,...
5,本科及以上,1-3年,面议,2018年11月06日,UI设计工程师,上海,上海华高股权投资管理有限公司,https://www.liepin.com/job/1915733868.shtml,https://www.liepin.com/company/9396717/,UI,100万以上
6,学历不限,1-3年,100-150k·15薪,2020年04月26日,UI设计师leader,,全球某知名信息科技公司,https://www.liepin.com/a/19017111.shtml,,UI,100万以上
7,统招本科,3-5年,160-180k·12薪,2020年04月23日,软件视觉设计师（UI）,长春,国内某知名企业,https://www.liepin.com/a/19928989.shtml,,UI,100万以上
8,统招本科,5-10年,100-200k·15薪,2020年04月21日,ui设计专家,,腾非信息科技有限公司,https://www.liepin.com/a/19379115.shtml,,UI,100万以上


Wall time: 32.4 s


# ⑩页面-分类-翻页

In [61]:
%%time
# 页面-参数模板翻页
r = session.get( url, params = payload)
list_df = list()
for k,v in 参数_keyword_UI_curPage.items():
    payload = v
    df = requests_liepin( url, params = payload)
    time.sleep(3+4*random())
    ## 备份
    #df.to_csv("homework/data_out/翻页_{key}_{k}.tsv"\
    #         .format(key=key, k=k), sep="\t", encoding="utf8")     
    df = df.assign (搜索关键词 = key)
    df = df.assign (所在页码 = k)    
    list_df.append(df)

df_all_curPage = pd.concat(list_df)
display(df_all_curPage)

#输出换index
df_all_curPage_2 = pd.concat(list_df).reset_index()
df_all_curPage_2.index = range(1,len(df_all_curPage) + 1)
df_all_curPage_2.index.name = '序'

#df_all_curPage_2.to_excel("翻页_UI_猎聘.xlsx",\
#                sheet_name="_".join(keywords))

,学历要求,工作经验,薪资,发布时间,职位名称,公司地点,公司名称,职位URL,公司URL,搜索关键词,所在页码
0,本科及以上,1-3年,6-10k·12薪,2020年04月23日,UI设计师,深圳,深圳市兰兹酒店管理有限公司,https://www.liepin.com/job/1927021149.shtml,https://www.liepin.com/company/9706919/,UI,0
1,大专及以上,经验不限,6-10k·12薪,2020年04月26日,UI设计,上海,艾瑞碧(上海)化妆品有限公司,https://www.liepin.com/job/1927592063.shtml,https://www.liepin.com/company/10197959/,UI,0
2,大专及以上,5-10年,20-30k·14薪,2020年04月24日,游戏主UI设计,上海,恺英网络,https://www.liepin.com/job/1927570189.shtml,https://www.liepin.com/company/5534889/,UI,0
3,统招本科,3-5年,面议,2020年04月26日,UI/UE设计师（临床）,北京,联仁健康医疗大数据科技股份有限公司,https://www.liepin.com/job/1927790263.shtml,https://www.liepin.com/company/12140561/,UI,0
4,大专及以上,经验不限,15-20k·12薪,2020年04月26日,游戏UI,,完美世界,https://www.liepin.com/job/1927788047.shtml,https://www.liepin.com/company/164236/,UI,0
...,...,...,...,...,...,...,...,...,...,...,...
35,大专及以上,1-3年,13-26k·15薪,2020年04月26日,UI设计师,上海,某大型科技互联网公司,https://www.liepin.com/a/19989993.shtml,,UI,9
36,统招本科,3-5年,15-30k·12薪,2020年04月26日,交互ui设计师,,某科技有限公司,https://www.liepin.com/a/19989417.shtml,,UI,9
37,学历不限,5-10年,25-40k·12薪,2020年04月26日,资深ui设计师,上海-徐汇区,比心陪练,https://www.liepin.com/a/19989013.shtml,,UI,9
38,学历不限,5-10年,25-40k·12薪,2020年04月26日,资深ui设计师,上海-徐汇区,比心陪练,https://www.liepin.com/a/19988981.shtml,,UI,9


Wall time: 59.9 s


# ⑪汇总以上数据价值表格

In [69]:
# 1、汇总公司和行业
df_firsttwo = pd.merge(df_all_company,df_all_industry)

# 2、汇总公司行业薪资
df_cis = pd.merge(df_firsttwo,df_all_salary)

# 3、加上翻页
# 汇总公司-行业-薪资-翻页：四个条件筛选
df_cis_curPage = df_cis.merge(df_all_curPage)
#输出换index
df_cis_curPage.reset_index()
df_cis_curPage.index = range(1,len(df_cis_curPage) + 1)
df_cis_curPage.index.name = '序'

# 输出Excel
with pd.ExcelWriter(r'H:\data mining\homework\猎聘_公司行业薪资分页_byV.xls') as writer:
    df_cis_curPage.to_excel(writer, sheet_name='公司行业薪资翻页_UI')
    df_all_company_2.to_excel(writer, sheet_name='公司分类_UI')
    df_all_industry_2.to_excel(writer, sheet_name='行业分类_UI')
    df_all_salary_2.to_excel(writer, sheet_name='薪资分类_UI')
    df_all_curPage_2.to_excel(writer, sheet_name='翻页_UI')